# Многонишково програмиране с Python

План на лекцията:
- Скалиране от 1 към много
- Какво е нишка ?
- [ ] Работа с нишки (Threads)
- [ ] Python GIL
- [ ] Threads, round 2
- [ ] "Многопроцесност"
- [ ] Бъдещето
- [ ] asyncio
- [ ] Примери
- [ ] Задачи

## Скалиране от 1 към много

Нека се върнем към едни по-прости времена - когато процесорът (да го наречем "v1") може да изпълнява една-единствена задача. Нека освен този прост процесор, да имаме и набор от задачи - `[t1, t2, t3]`.
Процесорът взима първата задача, изпълнява я, взема втората задача, изпълнява нея, и така докато не мине през всички задачи.

(Картинка, на която t1 -> t2 ---> t3 -->)

Нека си представим, че задачата `t2` отнема повече време, защото трябва да изчака за някакъв ресурс (например файлова операция). В това време, нашия процесор v1 чака, и не прави никакви изчисления. Това със сигурност не е оптимално.

### Какво е конкурентност ?

Нека да вземем един нов, по-добър процесор ("v2"), който може да разпознае кога даден процес чака за ресурс, и в такава ситуация да даде приоритет на друг процес. 
В тази ситуация, когато `t2` чака за ресурс, процесорът може да пусне `t3` по-рано. Когато ресурсът на `t2` стане наличен, процесорът ще спре `t3`, и ще даде време обратно на `t2`.   

(Картинка, на която t1 -> t2 -> t3 --> t2 ->)

Това изпълнение, ни позволява да изпълним задачите си по-бързо - вместо да чакаме `t2` да приключи, това време в чакане може да се използва за изпълнението на `t3`. 

Този модел на изпълнение, в който изпълнението на следващата задача не е нужно да изчаква приключването на текущата, се нарича "конкурентно изпълнение".
Важно е да се отбележи, че конкурентното изпълнение не е едновременно (паралелно) изпълнение на задачи - продължаваме да изпълняваме по една задача в даден момент, просто редът им не е детерминиран. Ние не знаем в какъв ред ще се изпълнят задачите и коя от тях ще бъде изпълнена първа.

### Какво е паралелизъм ?

Колкото и оптимално да е раздаването на процесорно време от нашия v2 процесор, той все още не може да прави паралелни изчисления. Затова нека въведем нашия "v3" процесор - той ще е два v2 процесора "залепени" един за друг. Това би означавало, че нашия v3 процесор може да изпълнява две задачи едновременно (паралелно).  

### Модерните процесори

Модерните процесори са изградени от няколко ядра (8, 10, 16), които могат да изпълняват различни задачи паралелно. В идеалния свят, ако нашата задача отнема `t` време на 1 ядро, то на `n` ядра, би трябвало да отнеме `t/n` време.

## Какво е нишка ?
- Какво е нишка принципно ?
- Зелени нишки


Нишката може да бъде разглеждана като най-малката смислена поредица от инструкции, които да бъдат изпълнени. Един процес съдържа една или повече нишки. 
Нишките могат да комуникират една с друга, с помощта на споделена памет или с т.нар. канали. 

Съществуват два вида нишки - нишки на ядрото (kernel threads) и потребителски нишки (user threads). Нишките на ядрото са менажирани от kernel-а - той се грижи за предаването на изпълнението от една нишка на друга. Kernel нишките споделят една и съща памет и файлови ресурси. Всяка нишка обаче има собствен стек, копие на регистрите и специална памет за конкретната нишка.

User нишките (наричани още "зелени" нишки, green threads) се управляват на ниво потребителски процес. Ядрото на ОС не знае за тяхното съществуване. Факта, че те се оправляват от потребителски процеси, прави предаването на контрола между всяка от тях евтина операция (от гледна точка на kernel-а, няма смяна на изпълняващата се нишка, съответно няма context switch). Главния недостатък на user нишките е при изпълнението на блокиращи операции (най-често свързани с IO операции) - при такава операция, вместо да се блокира само една нишка, се блокира целия процес. 

Съществуват три модела за работа с нишки, определяни от съотношението на брой kernel threads към user threads

- 1:1 - в този модел, на всяка нишка създадена от потребителя съотвества на kernel нишка
- N:1 - в този модел, всяка нишка създадена от потребителя се съпоставя към една kernel нишка
- M:N - в този модеел, M на брой потребителски нишки се съпоставят към N на брой kernel нишки

## Работа с нишки (Threads)
https://docs.python.org/3.10/library/threading.html?highlight=threading#module-threading
- Разглеждане на performance-а

За работа с нишки в Python използваме библиотеката `threading`. Тя ни предоставя различни инструменти за стартиране, синхронизиране и обща работа с нишки.

Основният клас с който ще работим, е класът `Thread`. Той има за цел да се погрижи за изпълнението на дадено парче питонски код, в отделна нишка. Можем да зададем кода, който искаме да изпълним по два начин - като `Callable` обект, подаден на конструктора на `Thread`, или като създадем собствен клас, който да наследи `Thread` и да презапишем `run` метода. 

За да стартираме нишка, трябва да извикаме `start` метода - той има за цел да подготви нова нишка, в която да бъде изпълнено съдържанието на `run` метода.

In [10]:
from threading import Thread

def hi():
    print("Thread with a Callable in the init")

t1 = Thread(target=hi)
t1.start()

class MyThread(Thread):
    def run(self):
        print("Custom Thread class")

t2 = MyThread()
t2.start()

Thread with a Callable in the init
Custom Thread class


Тук е важно да се отбележи, че когато наследяваме от `Thread`, не трябва да предефинираме никакви други методи, освен `__init__` и `run`. Можем да добавяме нови методи, но не трябва да променяме поведението на останалите методи в `Thread`. 

In [12]:
from threading import Thread
from time import sleep

def delayed_hi():
    sleep(2)
    print("Overslept a bit... hi")

t1 = Thread(target=delayed_hi)
t1.start()
print("The early bird gets the worm")

The early bird gets the worm


Overslept a bit... hi


В горния пример, изпълнението на програмата продължава напред със следващите редове код. Програмата ни приключва тогава, когато всички нишки за приключили работа.

Ако искаме да изчакаме дадена нишка да приключи преди да продължим с изпълнението на кода, можем да използваме метода `join`.

In [32]:
from threading import Thread
from time import sleep

def delayed_hi():
    sleep(1)
    print("I'm a thread")

t1 = Thread(target=delayed_hi)
t1.start()

print("The thread has started")
print("I want to wait for it to end")
t1.join()  # Comment this, to see the change
print("The thread has finished")

The thread has started
I want to wait for it to end
I'm a thread
The thread has finished


Методът `join` спира изпълненето на кода, докато нишката не приключи работата си или не срещне `Exception`. Възможно е да подадем `timeout` параметър на `join` - той индикира колко време да изчакаме нишката да приключи. Важно е да отбележим, че след изтичането на `timeout` параметъра, нишката може да продължи да работи. С методът `is_alive` можем да проверим дали дадена нишка все още работи.

In [34]:
from threading import Thread
from time import sleep 

def sleepy_thread():
    print(">Going to sleep")
    sleep(20)
    print(">Did a really good nap")

t1 = Thread(target=sleepy_thread)
t1.start()

print(f"Is the thread working ? {t1.is_alive()}")

print("Let's give it some time")
sleep(2)

print("Time to get up !")
t1.join(timeout=3)

print(f"Is the thread working ? {t1.is_alive()}")
t1.join()

print(f"What about now ? {t1.is_alive()}")

>Going to sleep
Is the thread working ? True
Let's give it some time
Time to get up !
Is the thread working ? True
>Did a really good nap
What about now ? False


Нека разгледаме следния практически пример - имаме следната функция, която изчислява (до някаква степен) безкрайна сума. Като пример, ще вземем следната сума  $$ \sum_{k=0}^{\infty} \frac{99^k}{(99(k+1))^{97}}  $$

Можем да напишеш функция, която изчислява част от реда - с начално $ k_{start} $ и крайно $ k_{end} $

In [34]:
import math

def calculate_partial_sum(start: int, end: int) -> int:
    sum = 0
    for k in range(start, end + 1):
        numerator = (99 ** k)
        denominator = (99 * (k+1)) ** 97
        sum += numerator // denominator
    
    return sum

Нека изчислим първите 25000 члена на този ред

In [50]:
import time

start = time.time()
calculate_partial_sum(0, 24000)
end = time.time()

print(f'The call took {(end-start):.2f} seconds')

The call took 13.06 seconds


Забелязваме, че кода ни отнема ~13 секунди да пресметне първите 24000 члена да реда. В този си вариант, членове на реда не зависят един от друг - това ни позволява да ги пресмятаме на интервали, които можем да пуснем на няколко нишки.

Нека като първи пример пуснем две нишки - едната да изчисли първите 12000 члена, а втората, останалите.

In [51]:
import time
from threading import Thread

t1 = Thread(target=calculate_partial_sum, args=(0, 12000))
t2 = Thread(target=calculate_partial_sum, args=(12001, 24000))

start = time.time()
t1.start()
t2.start()

while t1.is_alive() or t2.is_alive():
    time.sleep(0.1)

end = time.time()
print(f'The 2 threads took {(end-start):.2f} seconds')

The 2 threads took 14.82 seconds


Забелязваме, че на две нишки, кодът ни се изпълни за ~15 секунди - изненада. Нека пробваме на 4 нишки, каква е ситуацията.

In [52]:
import time
from threading import Thread

t1 = Thread(target=calculate_partial_sum, args=(0, 6000))
t2 = Thread(target=calculate_partial_sum, args=(6001, 12000))
t3 = Thread(target=calculate_partial_sum, args=(12001, 18000))
t4 = Thread(target=calculate_partial_sum, args=(18001, 24000))

start = time.time()
t1.start()
t2.start()
t3.start()
t4.start()

while t1.is_alive() or t2.is_alive() or t3.is_alive() or t4.is_alive():
    time.sleep(0.1)

end = time.time()
print(f'The 4 threads took {(end-start):.2f} seconds')

The 4 threads took 21.37 seconds


С 4 нишки, нещата стават още по-бавни... Какво се случва ?

## Python GIL

Една от особенностите на Python, е т.нар. global interpreter lock (GIL) - накратко, само една нишка може да работи с интерпретатора на Python в даден момент. 
Това е направено с цел, да се реши проблема с промяната на променлива от една нишка, докато друга я достъпва. (По-конкретно, проблемът може да се появи, когато дадена памет трябва да се освободи, след като в едната нишка се стига до порция от кода, в който променливата вече не се използва).

Съществуването на GIL в Python улеснява писането на код - нашия код винаги е thread-safe (няма как две нишки да се борят за един и същ ресурс). Освен това, GIL позволява използването на C библиотеки, които не са предвидени за работа в многонишков режим.

Наличието на GIL в Python обаче означава, че нашия "многонишков" код всъщност не е многонишков - да, ние стартираме няколко нишки, но те се изпълняват конкурентно, а не паралелно. На практика това означава, че при задачи, в които разчитаме единствено на операциите на процесора, кодът ни ще се държи все едно се изпълнява на една нишка. 

Това обаче е вярно за задачи, които са изцяло зависими от процесора - както казахме по-горе, понякога нашите задачи трябва да "чакат" за някакъв ресурс (например файл) - тогава интерпретатора ще отнеме контрола на чакащата нишка, и ще го предаде на следвщата нишка чакаща време за изпълнение. На практика това означава, че използването на нишки в Python би донесло подобрение във времето за изпълнение, независимо от GIL. 

Ситуации в които това би било вярно, е когато работим с GUI - през повечето време нашата програма няма да прави CPU изчисления, и активната нишка ще е тази, върху която се изпълнява графичния интерфейс. Друг пример, е когато имаме задачи с много IO операции - писане или четене от файлове.

Ако искате да научите повече детайли за GIL, може да разгледате [тази](https://realpython.com/python-gil/) статия, или [този](https://www.youtube.com/watch?v=Obt-vMVdM8s) клип.

## Threads, round 2
https://docs.python.org/3.10/library/threading.html?highlight=threading#module-threading
- Семафори
- Lock
- Condition
- Event

## "Многопроцесност"
https://docs.python.org/3.10/library/multiprocessing.html#module-multiprocessing

## Бъдещето
https://docs.python.org/3.10/library/concurrent.futures.html#module-concurrent.futures


## asyncio
https://docs.python.org/3.10/library/asyncio.html#module-asyncio